In [1]:
%%capture

%cd..

%load_ext autoreload
%autoreload 2

from IPython.display import display

In [2]:
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

In [3]:
def show_df(df: pd.DataFrame):
    display(df.head())
    print(df.shape)

In [4]:
import pickle

with open('/home/aris/projects/GEC_EHR/data/interim/proc_sunjae_data.pkl',
          'rb') as f:
    df_data = pickle.load(f)

show_df(df_data)

,text
./data/died_SOAP_dataset/001216365_20140930194420.txt,"abdominal discomfort, cough ?viral illness ver..."
./data/died_SOAP_dataset/001078162_20160924192147.txt,"Upper airway pain with breathing, ? due to inf..."
./data/died_SOAP_dataset/000918988_20151104184440.txt,PATIENT is status post sleeve gastrectomy.\nIt...
./data/died_SOAP_dataset/520138305_20161023164639.txt,"URI with viral pharyngitis, clear fluid in R m..."
./data/died_SOAP_dataset/001076295_20141222165037.txt,PATIENT is status post gastric bypass surgery....


(18866, 1)


In [5]:
from critic.step2_critic import *

In [7]:
model_name_list = [
    'bert-base-uncased',
    'roberta-base',
    'facebook/bart-base',
    'emilyalsentzer/Bio_ClinicalBERT',
    'dmis-lab/biobert-base-cased-v1.2',
]

model_name = model_name_list[1]

gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer = init_model(model_name)

Loaded gpt2 and roberta-base


In [8]:
example_sentence = 'I like eating apple.'

critic(gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer, example_sentence)

Bad! Your sentence log(p) = 12.655
Neighbor sentence with highest log(p): I like taking apple. (= 13.948)


(False, 12.655468940734863, ['I like taking apple.', 13.947656631469727])

In [9]:
example_sentence = 'I like taking apple.'

critic(gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer, example_sentence)

Bad! Your sentence log(p) = 13.948
Neighbor sentence with highest log(p): I liked taking apple. (= 14.494)


(False, 13.947656631469727, ['I liked taking apple.', 14.493749618530273])

In [10]:
example_sentence = 'I liked taking apple.'

critic(gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer, example_sentence)

Bad! Your sentence log(p) = 14.494
Neighbor sentence with highest log(p): I liked taking apples. (= 14.981)


(False, 14.493749618530273, ['I liked taking apples.', 14.981249809265137])

In [12]:
example_sentence = 'I liked having apple.'

critic(gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer, example_sentence)

Good! Your sentence log(p) = 15.066


(True, 15.065625190734863, None)

In [12]:
example_sentence = 'I only like eat apple\'s.'

critic(gpt2_model, gpt2_tokenizer, bert_model, bert_tokenizer, example_sentence)

Bad! Your sentence log(p) = 1.722
Neighbor sentence with highest log(p): So I only like eat apple's. (= 1.742)


(False, 1.7220458984375, ["So I only like eat apple's.", 1.7423502604166667])

Rank of 'I like eating apples', index 49

- gpt2: 1
- EleutherAI/gpt-neo-1.3B: 1
- emilyalsentzer/Bio_ClinicalBERT: 2
- dmis-lab/biobert-base-cased-v1.2: 4
- facebook/bart-base: 5
- bert-base-uncased: 9
- roberta-base: miss

- Fine-tuned gpt2: 1

In [9]:
example_sentence = 'I like eating apples.'

critic(model, tokenizer, example_sentence)

Good! Your sentence log(p) = -25.056


(True, -25.0556640625, None)

In [8]:
_c, _i, _a, _e = 0, 0, 0,0

for row in df_data.iterrows():
    # print(i[0])
    # print(i[1])
    _path, _text = row
    _text = _text['text']

    sentences = _text.split('\n')

    # print(sentences)
    for sent in sentences:

        try:
            is_good, score, counter_example = critic(
        gpt2_model,gpt2_tokenizer,bert_model,bert_tokenizer, sent,
                                                 verbose=False)
        except TypeError:
            _e+=1
            _a+=1

        if not is_good:
            print('-' * 50)
            print(f'{_i}. Sentence:        {sent}')
            print(f'{_i}. Counter example: {counter_example[0]}')
            _i += 1
        else:
            print('-' * 50)
            print(f'Sentence:              {sent}')
        _a += 1

    if _c > 5:
        break
    _c += 1

print('*' * 50)
print(f'Tested {_a} sentences.')
print(f'{_i} corrections made')
print(f'{_e} errors')

--------------------------------------------------
Sentence:              abdominal discomfort, cough ?viral illness versus mild constipation ?reflux related
--------------------------------------------------
0. Sentence:        Upper airway pain with breathing, ? due to inflammation vs. muscle tightness from running, +/- element of bronchospasm
0. Counter example: Upper airway pain with breathing,? due to inflammation vs. muscle tightness from running, + /- elements of bronchospasm
--------------------------------------------------
1. Sentence:        PATIENT is status post sleeve gastrectomy.
1. Counter example: PATIENT is to status post sleeve gastrectomy.
--------------------------------------------------
2. Sentence:        It has been 6 months since her surgery.
2. Counter example: , It has been 6 months since her surgery.
--------------------------------------------------
Sentence:              PATIENT is doing an excellent job.
--------------------------------------------------

In [43]:
good, bad = [], []
errors = []

for row in df_data.iterrows():
    _path, _text = row
    _text = _text['text']

    sentences = _text.split('\n')

    # print(sentences)
    for sent in sentences:
        try:
            is_good, score, counter_example = gpt2_critic(sent, verbose=False)

            if not is_good:
                bad.append((_path, sent, counter_example[0]))
            else:
                good.append((_path, sent))
        except Exception:
            errors.append((_path, sent))

pp, pn = len(good), len(bad)

print(f'Tested {pp + pn} sentences.')
print(f'{pn} corrections made')
print(f'correction rate: {pn / (pp + pn):.2}')
print(f'{len(errors)} errors encountered.')

Tested 105421 sentences.
38421 corrections made
correction rate: 0.36
231 errors encountered.


In [47]:
# Bug

good[42]

('./data/died_SOAP_dataset/000739056_20140722002949.txt',
 'PATIENT is status post sleeve gastrectomy and it has been 3 weeks since her surgery.\nPATIENT has been mostly compliant with her diet thus far.\nShe is very nearly meeting her protein needs with appropriate items, although she did add in some high sugar syrup in order to make her protein drinks more palatable.\nShe does understand that this is not recommended due to the high calorie and sugar content.\nOtherwise, she is doing well and she has no complaints at this time.\nShe seems willing to listen to all of the education that was provided today and I have no concerns about her moving forward.\n\nPATIENT was provided with stage IV and stage V diet education material.\nShe understands that for the next month, she will be on a stage IV bariatric diet, which consists of all high protein foods.\nShe should not be having any fruits, vegetables or foods high in carbohydrates at this point.\nI would like her to have 3 meals and 3 sna

In [46]:
bad[0]

('./data/died_SOAP_dataset/001078162_20160924192147.txt',
 'Upper airway pain with breathing, ? due to inflammation vs. muscle tightness from running, +/- element of bronchospasm',
 'Upper airway pain with breathing,? due to inflammation vs. muscle tightness from running, + /- elements of bronchospasm')

In [40]:
with open('/home/aris/projects/GEC_EHR/data/interim/200_ap.txt', 'r') as f:
    ap_200_sents = f.read().split('\n')[:-1]

len(ap_200_sents)

200

In [48]:
correct_ratio = {}

for _model_name in model_name_list:
    model, tokenizer = init_model(_model_name)

    _c, _e = 0, 0
    for sent in ap_200_sents:

        try:
            is_good, _, _ = critic(model, tokenizer, sent, verbose=False)
        except TypeError as err:
            _e += 1
            continue

        if not is_good:
            _c += 1

    correct_ratio[_model_name] = _c, _e

Loaded gpt2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded bert-base-uncased
Loaded roberta-base
Loaded facebook/bart-base


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded emilyalsentzer/Bio_ClinicalBERT


Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loaded dmis-lab/biobert-base-cased-v1.2
Loaded EleutherAI/gpt-neo-1.3B


In [49]:
for _model_name, (_c, _e) in correct_ratio.items():
    _a = 200 - _e

    print('-' * 50)
    print(_model_name)
    print(f'Corrected {_c}')
    print(f'Errored {_e}')
    print(f'Corrected ratio: {_c / _a:.2}')

--------------------------------------------------
gpt2
Corrected 88
Errored 0
Corrected ratio: 0.44
--------------------------------------------------
bert-base-uncased
Corrected 194
Errored 0
Corrected ratio: 0.97
--------------------------------------------------
roberta-base
Corrected 199
Errored 0
Corrected ratio: 0.99
--------------------------------------------------
facebook/bart-base
Corrected 200
Errored 0
Corrected ratio: 1.0
--------------------------------------------------
emilyalsentzer/Bio_ClinicalBERT
Corrected 199
Errored 1
Corrected ratio: 1.0
--------------------------------------------------
dmis-lab/biobert-base-cased-v1.2
Corrected 199
Errored 1
Corrected ratio: 1.0
--------------------------------------------------
EleutherAI/gpt-neo-1.3B
Corrected 78
Errored 0
Corrected ratio: 0.39


In [52]:
model, tokenizer = init_model('gpt2')

_i = 0
for sent in ap_200_sents:

    try:
        is_good, _, counter_example = critic(model, tokenizer, sent, verbose=False)
    except TypeError as err:
        continue

    if not is_good:
        print('-' * 50)
        print(f'{_i}. Sentence:        {sent}')
        print(f'{_i}. Counter example: {counter_example[0]}')
        _i += 1

Loaded gpt2
--------------------------------------------------
0. Sentence:        I do believe he is probably eating a lot more than he realizes at this point.
0. Counter example: I do believe he is probably taking a lot more than he realizes at this point.
--------------------------------------------------
1. Sentence:        PATIENT was given these specific goals today, as well as Smart Snacking handout and a Plate Method handout:
1. Counter example: PATIENT was given these specific goals today, as well as Smart Snacking handouts and a Plate Method handout:
--------------------------------------------------
2. Sentence:        Normal examination. Mild hypochromic microcytic indices, but no significant anemia, unclear etiology.
2. Counter example: Normal examination. Mild hypochromic microcytic indices, but no significant anemia, clear etiology.
--------------------------------------------------
3. Sentence:        She asked followup questions to the education that was provided.
3. C

In [17]:
ids = tokenizer(example_sentence)['input_ids']

ids

[101, 1045, 2066, 5983, 6207, 1012, 102]

In [20]:
ids[1] = tokenizer.mask_token_id

ids

[103, 1045, 2066, 5983, 6207, 1012, 102]

In [21]:
tokenizer.decode(ids)

'[MASK] i like eating apple. [SEP]'